## 自动求导Automatic Differentiation with torch.autograd
在训练神经网络时，最常用的算法是 反向传播。在该算法中，根据损失函数相对于给定参数的梯度来调整参数（模型权重） 。

为了计算这些梯度，PyTorch 有一个名为的内置微分引擎torch.autograd。它支持自动计算任何计算图的梯度。

考虑最简单的单层神经网络，其输入为x，参数为w和b，以及一些损失函数。它可以在 PyTorch 中按以下方式定义：

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True) # 权重参数 - 需要计算梯度
b = torch.randn(3, requires_grad=True) # 偏置参数 - 需要计算梯度
# 前向传播 - 构建计算图
z = torch.matmul(x, w)+b # 计算图节点1
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y) # 计算图节点2

### requires_grad=True 详解
requires_grad=True 是 PyTorch 中一个非常重要的参数属性，用于控制张量是否需要计算和跟踪梯度。
#### 基本概念
requires_grad 是 PyTorch 张量的一个布尔属性，决定该张量是否需要参与自动微分计算：
requires_grad=True：张量需要计算梯度，会跟踪所有在其上的操作
requires_grad=False：张量不需要计算梯度，不会跟踪操作（默认值）
#### 为什么需要设置 requires_grad=True？
参数优化：神经网络的权重(w)和偏置(b)是需要在训练过程中优化的参数
梯度计算：只有设置了 requires_grad=True 的张量，PyTorch 才会为其计算梯度
反向传播：在反向传播过程中，只有这些张量的梯度会被计算和更新
#### 当 requires_grad=True 时：
PyTorch 会构建计算图，跟踪所有对张量的操作
在调用 .backward() 时，会自动计算梯度
梯度存储在张量的 .grad 属性中

#### 相关属性和方法
.grad：存储计算得到的梯度
.grad_fn：指向创建该张量的函数（计算图节点）
.backward()：执行反向传播计算梯度
torch.no_grad()：临时禁用梯度计算

## Computing Gradients

In [3]:
# 反向传播 - 自动求导
loss.backward()
print(w.grad)# 权重的梯度
print(b.grad) # 偏置的梯度

tensor([[0.1645, 0.0804, 0.0124],
        [0.1645, 0.0804, 0.0124],
        [0.1645, 0.0804, 0.0124],
        [0.1645, 0.0804, 0.0124],
        [0.1645, 0.0804, 0.0124]])
tensor([0.1645, 0.0804, 0.0124])


我们只能获取grad计算图的叶节点的属性，这些节点的requires_grad属性设置为True。对于图中的所有其他节点，梯度将不可用。

出于性能考虑，我们只能 backward在给定的图上执行一次梯度计算。如果我们需要backward在同一张图上执行多次调用，则需要将 传递 retain_graph=True给backward调用函数。

#### 自动求导的核心组件
1. 计算图（Computation Graph）
》每个操作都会创建一个计算图节点，记录操作类型和依赖关系。

In [4]:
print(f"z 的梯度函数: {z.grad_fn}")        # <AddBackward0 object>
print(f"loss 的梯度函数: {loss.grad_fn}")  # <BinaryCrossEntropyWithLogitsBackward0 object>

z 的梯度函数: <AddBackward0 object at 0x10e547d30>
loss 的梯度函数: <BinaryCrossEntropyWithLogitsBackward0 object at 0x10e5655a0>


2. 叶节点（Leaf Nodes）
》有叶节点（如模型参数）才能获得 .grad 属性

In [5]:
# 叶节点是可以计算梯度的张量
print(f"w 是叶节点: {w.is_leaf}")      # True
print(f"w 需要梯度: {w.requires_grad}") # True

# 中间节点通常不是叶节点
print(f"z 是叶节点: {z.is_leaf}")      # False
print(f"z 需要梯度: {z.requires_grad}") # True


w 是叶节点: True
w 需要梯度: True
z 是叶节点: False
z 需要梯度: True


3. 梯度计算

In [6]:
# 执行反向传播
loss.backward()

# 梯度存储在 .grad 属性中
print("权重梯度:", w.grad.shape)  # torch.Size([5, 3])
print("偏置梯度:", b.grad.shape)  # torch.Size([3])


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

## Disabling Gradient Tracking
默认情况下，所有带有requires_grad=True的张量都会跟踪其计算历史并支持梯度计算。然而，在某些情况下我们不需要这样做，例如，当我们已经训练好模型，只想将其应用于一些输入数据时，即我们只想通过网络 进行前向torch.no_grad()计算。我们可以通过在计算代码周围添加以下代码块来停止跟踪计算：

In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

实现相同结果的另一种方法是使用detach()张量上的方法：

In [ ]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

可能希望禁用梯度跟踪的原因如下：
将神经网络中的某些参数标记为冻结参数。

当您仅进行前向传递时，可以加快计算速度，因为不跟踪梯度的张量的计算会更有效率。

## 自动求导的两种模式
1. 训练模式（需要梯度）

In [ ]:
# 默认模式 - 跟踪计算历史
w = torch.randn(5, 3, requires_grad=True)
z = torch.matmul(x, w) + b
print(z.requires_grad)  # True

2. 推理模式（不需要梯度）

In [ ]:
# 方法1: 使用 torch.no_grad() 上下文管理器
with torch.no_grad():
    z = torch.matmul(x, w) + b
    print(z.requires_grad)  # False

# 方法2: 使用 detach() 方法
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)  # False

# 方法3: 禁用全局梯度计算
torch.set_grad_enabled(False)
z = torch.matmul(x, w) + b
print(z.requires_grad)  # False

torch.set_grad_enabled(True)  # 恢复

In [7]:
import torch
import torch.nn as nn

# 定义简单神经网络
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(5, 3)
    
    def forward(self, x):
        return self.linear(x)

# 创建模型和数据
model = SimpleNet()
x = torch.randn(2, 5)
y_true = torch.randn(2, 3)

# 前向传播
y_pred = model(x)
loss = nn.MSELoss()(y_pred, y_true)

# 自动求导
loss.backward()

# 查看参数梯度
for name, param in model.named_parameters():
    print(f"{name} 的梯度形状: {param.grad.shape}")


linear.weight 的梯度形状: torch.Size([3, 5])
linear.bias 的梯度形状: torch.Size([3])
